In [1]:
#Habilitar intellisense
%config IPCompleter.greedy=True

## Enunciado
La prueba consiste en completar el código marcado con **#FIXME** para obtener automáticamente el resumen de un texto.

La decripción general de las reglas/algoritmo:

- Identificar las frecuencias de las palabras en el texto.
- Obtener las oraciones/párrafos en el texto.
- Obtener un score para cada oración/párrafo que indique su importancia en el texto, tendrán mayor score aquellas oraciones que incluyan las palabras con mayor frecuencia.
- Seleccionar las 3 oraciones con el score más alto para construir el resumen.

In [2]:
import urllib3
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from string import punctuation
from collections import defaultdict
import collections
from heapq import nlargest
import unicodedata

http = urllib3.PoolManager()

ModuleNotFoundError: No module named 'nltk'

In [189]:
articleURL = "https://www.muyinteresante.es/naturaleza/video/objetivo-2030-proteger-el-30-de-la-tierra-751579255112"

In [190]:
import requests
respuesta = requests.get(articleURL) 
soup = BeautifulSoup(respuesta.text,"html.parser")

In [191]:
text_containers = soup.body.find_all(name='div', class_='paragraph--text')

In [211]:
#El valor de la variable text debe ser una cadena con los párrafos separados por '\n'
text = '';
for tc in text_containers:
    p_containers = tc.find_all(name='p')
    for p in p_containers:
        text = text + p.text + '\n'
text = text[26:]
text

'El Convenio de la Organización de las Naciones Unidas (ONU) sobre Diversidad Biológica acaba de presentar el primer borrador de lo que será su hoja de ruta para los próximos diez años. El objetivo es tan ambicioso como necesario: proteger cerca de un tercio de la superficie total del planeta, incluyendo ecosistemas tanto terrestres como acuáticos.\nEn 2019, la Plataforma Intergubernamental sobre la Biodiversidad y los Servicios Ecosistémicos (IPBES) presentaba un demoledor informe sobre el estado de la biodiversidad a nivel mundial que indicaba que en el mundo hay en torno a un millón de especies en peligro, y que las tasas de extinción se están acelerando a un ritmo sin precedentes como consecuencia de las actividades humanas. Algunos expertos ya consideran, de hecho, de que estamos viviendo la sexta extinción masiva.\nEsta enorme pérdida de biodiversidad tiene consecuencias graves para los ecosistemas, pero también para el bienestar humano. Por ejemplo, un artículo publicado en la r

In [215]:
text = unicodedata.normalize("NFKD",text) #replace \xa0 with regular space
print(text)

El Convenio de la Organización de las Naciones Unidas (ONU) sobre Diversidad Biológica acaba de presentar el primer borrador de lo que será su hoja de ruta para los próximos diez años. El objetivo es tan ambicioso como necesario: proteger cerca de un tercio de la superficie total del planeta, incluyendo ecosistemas tanto terrestres como acuáticos.
En 2019, la Plataforma Intergubernamental sobre la Biodiversidad y los Servicios Ecosistémicos (IPBES) presentaba un demoledor informe sobre el estado de la biodiversidad a nivel mundial que indicaba que en el mundo hay en torno a un millón de especies en peligro, y que las tasas de extinción se están acelerando a un ritmo sin precedentes como consecuencia de las actividades humanas. Algunos expertos ya consideran, de hecho, de que estamos viviendo la sexta extinción masiva.
Esta enorme pérdida de biodiversidad tiene consecuencias graves para los ecosistemas, pero también para el bienestar humano. Por ejemplo, un artículo public

In [250]:
def summarize(text, n):
    sentences = sent_tokenize(text)
    assert n <= len(sentences)
    words = word_tokenize(text.lower())
    
    our_stopwords = set(stopwords.words('spanish') + list(punctuation) + ['“', '”'])
    
    words_without_stops=[word for word in words if word not in our_stopwords]
    
    #obtener un diccionario con la frecuencia de las palabras en words_without_stops
    word_frequencies = collections.Counter(words_without_stops)
    
    #Obtener los scores para cada oración de acuerdo a la reglas anteriormente descritas
    #sentence_scores es un diccionario con la posición/indice de cada párrafo y su score
    
    sentence_scores = defaultdict(int)
    for i, sentence in enumerate(sentences):
        peso = 0
        for word in word_tokenize(sentence.lower()):
            peso = peso + word_frequencies[word]
        sentence_scores[i] = peso

    sorted_sentence_scores = sorted(sentence_scores.items(), key=lambda item: item[1], reverse=True)
    sentences_with_score = [{'index': k, 'score': v} for k, v in sorted_sentence_scores]
    
    best_sentences = sentences_with_score[:n]
    indexes_of_the_highest_scores = [s['index'] for s in best_sentences]
    
    summary_sentences = []
    for index in indexes_of_the_highest_scores:
        summary_sentences.append(sentences[index])
    return summary_sentences

In [251]:
summarize(text,3)

['El denominado “borrador preliminar del marco mundial de la diversidad biológica posterior a 2020” que acaba de ser presentado por la ONU incluye veinte metas orientadas a la acción para 2030 orientadas a objetivos como reducir las amenazas a la biodiversidad y satisfacer las necesidades de las personas a través de la utilización sostenible y la participación en los beneficios.',
 '“Se da el hecho que en los procesos de toma de decisiones a menudo los representantes de los Estados son personas de perfil político con un conocimiento técnico escaso de la situación de la biodiversidad y las consecuencias de no actuar, cuando no son representantes de Ministerios de asuntos exteriores, economía o comercio de sus respectivos países”, reflexionaba  Jaume Grau López, biólogo y Coordinador del Área de Conservación de la Naturaleza de Ecologistas en Acción, en una artículo publicado en 2013 en la revista Ecología Política.',
 'En octubre de 2020 tendrá lugar en Kunming (China